In [ ]:
!pip install ImageHash

In [ ]:
from PIL import Image
from google.colab import drive
import imagehash
import os
import sys
from PIL import Image, ImageStat, ImageOps,ImageEnhance
import cv2
import zipfile
import math
import shutil
import random
from collections import defaultdict

In [ ]:
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    # specify the path of the folder containing "file_name" by changing the lecture index:
    path_to_file = '/content/gdrive/MyDrive/BT5153'
    print(path_to_file)
    # change current path to the folder containing "file_name"
    os.chdir(path_to_file)
    !pwd

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/BT5153
/content/gdrive/.shortcut-targets-by-id/1-EuPWwH26BHpNG41utRuC3Oo7qwXYRcD/BT5153


In [ ]:
# Counting the numbers we have for each file
dog_label_folder_raw = path_to_file+'/dog label/Raw'

for folder_name in os.listdir(dog_label_folder_raw):
    folder_path = os.path.join(dog_label_folder_raw, folder_name)
    if os.path.isdir(folder_path):
        image_count = len([file for file in os.listdir(folder_path)
                   if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))])
        print(f"Folder '{folder_name}' contains {image_count} images.")

Folder 'Happy' contains 68 images.
Folder 'Angry' contains 20 images.
Folder 'Sad' contains 18 images.
Folder 'Scared' contains 6 images.
Folder 'Not sure' contains 0 images.
Folder 'Bored (sad?)' contains 17 images.


In [ ]:
def handle_zip_file(zip_path, temp_folder):
    # Create the temporary folder
    if not os.path.exists(temp_folder):
        os.makedirs(temp_folder)

    # Extract the zip file into the temporary folder
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(temp_folder)

    return temp_folder

def delete_temp_folder(temp_folder):
    if os.path.exists(temp_folder):
        shutil.rmtree(temp_folder)

<h1>Finding Duplicates</h1>

In [ ]:
def find_duplicates(dog_label_folder):
    hashes = {}
    duplicates = []
    for folder_name in os.listdir(dog_label_folder):
        folder_path = os.path.join(dog_label_folder, folder_name)
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                    image_path = os.path.join(folder_path, filename)
                    hash = imagehash.average_hash(Image.open(image_path))
                    if hash in hashes:
                        duplicates.append((folder_name, filename, hashes[hash]))
                    else:
                        hashes[hash] = (folder_name, filename)
    return duplicates

duplicates = find_duplicates(dog_label_folder_raw)
print(f"Found {len(duplicates)} duplicate images.")

Found 0 duplicate images.


<h1>Checking Image Quality</h1>

In [ ]:
def is_overexposed(image_path, threshold=200):
    img = Image.open(image_path)
    stat = ImageStat.Stat(img)
    return stat.mean[0] > threshold

def is_underexposed(image_path, threshold=50):
    img = Image.open(image_path)
    stat = ImageStat.Stat(img)
    return stat.mean[0] < threshold

for folder_name in os.listdir(dog_label_folder_raw):
    folder_path = os.path.join(dog_label_folder_raw, folder_name)
    if os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
          if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
              image_path = os.path.join(folder_path, filename)
              if is_overexposed(image_path):
                  print(f"{image_path} is overexposed")
              if is_underexposed(image_path):
                  print(f"{image_path} is underexposed")
    print(f"Checked {folder_name}")

Checked Happy
Checked Angry
Checked Sad
Checked Scared
Checked Not sure
Checked Bored (sad?)


In [ ]:
def is_overexposed(image_path, threshold=200):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return cv2.mean(gray)[0] > threshold

def is_underexposed(image_path, threshold=50):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return cv2.mean(gray)[0] < threshold

for folder_name in os.listdir(dog_label_folder_raw):
    folder_path = os.path.join(dog_label_folder_raw, folder_name)
    if os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
            if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                image_path = os.path.join(folder_path, filename)
                if is_overexposed(image_path):
                    print(f"{image_path} is overexposed")
                if is_underexposed(image_path):
                    print(f"{image_path} is underexposed")


    print(f"Checked {folder_name}")

Checked Happy
Checked Angry
Checked Sad
Checked Scared
Checked Not sure
Checked Bored (sad?)


In [ ]:
def is_overexposed(image_path, threshold=200):
    img = Image.open(image_path)
    stat = ImageStat.Stat(img)
    return stat.mean[0] > threshold

def is_underexposed(image_path, threshold=50):
    img = Image.open(image_path)
    stat = ImageStat.Stat(img)
    return stat.mean[0] < threshold

# Temporary folder to extract .zip files
temp_folder = "temp_extracted"

# Ensure the temp folder exists
os.makedirs(temp_folder, exist_ok=True)

# Iterate through the files in the folder
for file_name in os.listdir(dog_label_folder_raw):
    file_path = os.path.join(dog_label_folder_raw, file_name)

    # Handle the zip file
    if file_name.endswith('.zip'):
      unzipped_folder = handle_zip_file(os.path.join(dog_label_folder_raw, file_name), temp_folder)
      print(f"Extracted {file_name} to {temp_folder}")

# Process the extracted files
for folder_name in os.listdir(temp_folder):
    folder_path = os.path.join(temp_folder, folder_name)
    if os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
            if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                image_path = os.path.join(folder_path, filename)
                if is_overexposed(image_path):
                    print(f"{image_path} is overexposed")
                if is_underexposed(image_path):
                    print(f"{image_path} is underexposed")
        print(f"Checked {folder_name}")

# Clean up: Delete the temporary folder and its contents
delete_temp_folder(temp_folder)
print(f"Deleted temporary folder: {temp_folder}")

Deleted temporary folder: temp_extracted


<h1>Checking the ratio of the images</h1>

In [ ]:
def get_aspect_ratio(image_path):
    with Image.open(image_path) as img:
        width, height = img.size
        gcd = math.gcd(width, height)
        ratio = (width//gcd, height//gcd)
        return ratio

# for folder_name in os.listdir(os.path.join(dog_label_folder_raw, 'Raw')):
for folder_name in os.listdir(dog_label_folder_raw):
    folder_path = os.path.join(os.path.join(dog_label_folder_raw, folder_name))
    if os.path.isdir(folder_path):
        print(f"Checking aspect ratios for images in folder: {folder_name}")
        ratios = {}
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                image_path = os.path.join(folder_path, filename)
                ratio = get_aspect_ratio(image_path)
                if ratio in ratios:
                    ratios[ratio] += 1
                else:
                    ratios[ratio] = 1
        print(f"Aspect ratios found in {folder_name}:")
        for ratio, count in ratios.items():
            print(f"{ratio}: {count} images")
        print("\n\n")

Checking aspect ratios for images in folder: Happy
Aspect ratios found in Happy:
(348, 523): 1 images
(1108, 1477): 10 images
(4, 3): 6 images
(3, 4): 30 images
(1179, 1769): 2 images
(554, 739): 1 images
(3, 2): 1 images
(9, 16): 7 images
(1179, 1570): 1 images
(480, 853): 1 images
(449, 778): 1 images
(511, 904): 1 images
(1179, 1474): 1 images
(407, 641): 1 images
(309, 463): 1 images
(195, 422): 1 images
(131, 215): 1 images
(540, 787): 1 images



Checking aspect ratios for images in folder: Angry
Aspect ratios found in Angry:
(3, 4): 11 images
(480, 853): 1 images
(492, 703): 1 images
(9, 16): 5 images
(195, 422): 1 images
(1, 1): 1 images



Checking aspect ratios for images in folder: Sad
Aspect ratios found in Sad:
(277, 369): 1 images
(3, 4): 11 images
(9, 16): 2 images
(207, 448): 1 images
(1108, 1477): 2 images
(1179, 884): 1 images



Checking aspect ratios for images in folder: Scared
Aspect ratios found in Scared:
(1179, 2038): 1 images
(3, 4): 5 images



Checking aspec

<h1>Playing with the images</h1>

In [ ]:
from PIL import Image
import os

Remove Existed Image in Train/Test folder

In [ ]:
# #Clean all image in the clean folder
# train_folder = 'dog label/clean/train'
# test_folder = 'dog label/clean/test'

# for filename in os.listdir(train_folder):
#     file_path = os.path.join(train_folder, filename)
#     if os.path.isfile(file_path):
#         os.remove(file_path)
#         print(f"Deleted: {file_path}")

# for filename in os.listdir(test_folder):
#     file_path = os.path.join(test_folder, filename)
#     if os.path.isfile(file_path):
#         os.remove(file_path)
#         print(f"Deleted: {file_path}")

In [ ]:
target_ratio = (224,224) # Change it!

Generate "Angry" image

In [ ]:
# dog_label_folders_angry = ['/content/gdrive/MyDrive/BT5153/dog label/Raw/Angry', '/content/gdrive/MyDrive/BT5153/dog label/Raw/Angry/angry1865']

# def random_rotate(img, i):
#     angle = [90, 180, 270]
#     return img.rotate(angle[i])

# def random_flip(img, i):
#     flip_type = [Image.FLIP_LEFT_RIGHT, Image.FLIP_TOP_BOTTOM, Image.TRANSPOSE]
#     return img.transpose(flip_type[i])

# augmentations = [random_rotate, random_flip]

# counter = 0

# for folder_name in dog_label_folders_angry:
#     for filename in os.listdir(folder_name):
#         if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
#             label = 'angry'

#             img_path = os.path.join(folder_name, filename)
#             img = Image.open(img_path)

#             # Resize
#             img_ratio = img.size[0] / img.size[1]
#             target_ratio_val = target_ratio[0] / target_ratio[1]

#             if img_ratio > target_ratio_val:
#                 new_width = target_ratio[0]
#                 new_height = int(new_width / img_ratio)
#             else:
#                 new_height = target_ratio[1]
#                 new_width = int(new_height * img_ratio)

#             img = img.resize((new_width, new_height))

#             # Pad
#             padded_img = Image.new('RGB', target_ratio)
#             padded_img.paste(img, ((target_ratio[0]-new_width)//2, (target_ratio[1]-new_height)//2))

#             # Save original padded image
#             padded_img.save(os.path.join(train_folder, f"{label}_{counter}.jpg"))
#             counter += 1

#             # Augment image 3 times
#             for i in range(3):
#                 augmented_img = random.choice(augmentations)(padded_img, i)
#                 augmented_img.save(os.path.join(train_folder, f"{label}_{counter}.jpg"))
#                 counter += 1

#             if counter % 500 == 0:
#                 print(f"{counter} angry images generated!")


500 angry images generated!
1000 angry images generated!
1500 angry images generated!
2000 angry images generated!
2500 angry images generated!
3000 angry images generated!
3500 angry images generated!
4000 angry images generated!
4500 angry images generated!
5000 angry images generated!
5500 angry images generated!
6000 angry images generated!
6500 angry images generated!
7000 angry images generated!
7500 angry images generated!


Generate "Sad" image

In [ ]:
# dog_label_folders_sad = ['/content/gdrive/MyDrive/BT5153/dog label/Raw/Sad', '/content/gdrive/MyDrive/BT5153/dog label/Raw/Sad/sad1000']

# def random_rotate(img, i):
#     angle = [90, 180, 270]
#     return img.rotate(angle[i])

# def random_flip(img, i):
#     flip_type = [Image.FLIP_LEFT_RIGHT, Image.FLIP_TOP_BOTTOM, Image.TRANSPOSE]
#     return img.transpose(flip_type[i])

# augmentations = [random_rotate, random_flip]

# counter = 0

# for folder_name in dog_label_folders_sad:
#     for filename in os.listdir(folder_name):
#         if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
#             label = 'sad'

#             img_path = os.path.join(folder_name, filename)
#             img = Image.open(img_path)

#             # Resize
#             img_ratio = img.size[0] / img.size[1]
#             target_ratio_val = target_ratio[0] / target_ratio[1]

#             if img_ratio > target_ratio_val:
#                 new_width = target_ratio[0]
#                 new_height = int(new_width / img_ratio)
#             else:
#                 new_height = target_ratio[1]
#                 new_width = int(new_height * img_ratio)

#             img = img.resize((new_width, new_height))

#             # Pad
#             padded_img = Image.new('RGB', target_ratio)
#             padded_img.paste(img, ((target_ratio[0]-new_width)//2, (target_ratio[1]-new_height)//2))

#             # Save original padded image
#             padded_img.save(os.path.join(train_folder, f"{label}_{counter}.jpg"))
#             counter += 1

#             # Augment image 3 times
#             for i in range(3):
#                 augmented_img = random.choice(augmentations)(padded_img, i)
#                 augmented_img.save(os.path.join(train_folder, f"{label}_{counter}.jpg"))
#                 counter += 1

#             if counter % 500 == 0:
#                 print(f"{counter} sad images generated!")


500 sad images generated!
1000 sad images generated!
1500 sad images generated!
2000 sad images generated!
2500 sad images generated!
3000 sad images generated!
3500 sad images generated!
4000 sad images generated!


Generate "Happy" image

In [ ]:
# dog_label_folders_happy = ['/content/gdrive/MyDrive/BT5153/dog label/Raw/Happy', '/content/gdrive/MyDrive/BT5153/dog label/Raw/Happy/happy1865']

# def random_rotate(img, i):
#     angle = [90, 180, 270]
#     return img.rotate(angle[i])

# def random_flip(img, i):
#     flip_type = [Image.FLIP_LEFT_RIGHT, Image.FLIP_TOP_BOTTOM, Image.TRANSPOSE]
#     return img.transpose(flip_type[i])

# augmentations = [random_rotate, random_flip]

# counter = 0

# for folder_name in dog_label_folders_happy:
#     for filename in os.listdir(folder_name):
#         if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
#             label = 'happy'

#             img_path = os.path.join(folder_name, filename)
#             img = Image.open(img_path)

#             # Resize
#             img_ratio = img.size[0] / img.size[1]
#             target_ratio_val = target_ratio[0] / target_ratio[1]

#             if img_ratio > target_ratio_val:
#                 new_width = target_ratio[0]
#                 new_height = int(new_width / img_ratio)
#             else:
#                 new_height = target_ratio[1]
#                 new_width = int(new_height * img_ratio)

#             img = img.resize((new_width, new_height))

#             # Pad
#             padded_img = Image.new('RGB', target_ratio)
#             padded_img.paste(img, ((target_ratio[0]-new_width)//2, (target_ratio[1]-new_height)//2))

#             # Save original padded image
#             padded_img.save(os.path.join(train_folder, f"{label}_{counter}.jpg"))
#             counter += 1

#             # Augment image 3 times
#             for i in range(3):
#                 augmented_img = random.choice(augmentations)(padded_img, i)
#                 augmented_img.save(os.path.join(train_folder, f"{label}_{counter}.jpg"))
#                 counter += 1

#             if counter % 500 == 0:
#                 print(f"{counter} happy images generated!")


500 happy images generated!
1000 happy images generated!
1500 happy images generated!
2000 happy images generated!
2500 happy images generated!
3000 happy images generated!
3500 happy images generated!
4000 happy images generated!
4500 happy images generated!
5000 happy images generated!
5500 happy images generated!
6000 happy images generated!
6500 happy images generated!
7000 happy images generated!
7500 happy images generated!


<h1>Let's go</h1>

In [ ]:
# Steal some to put into test
# train_dir = 'dog label/clean/train'
# test_dir = 'dog label/clean/test'
# test_split = 0.2


# os.makedirs(test_dir, exist_ok=True)
# train_files = [f for f in os.listdir(train_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
# num_test_files = int(len(train_files) * test_split)
# test_files = random.sample(train_files, num_test_files)
# for file in test_files:
#     shutil.move(os.path.join(train_dir, file), os.path.join(test_dir, file))


In [ ]:
# train_dir = 'dog label/clean/train'
# test_dir = 'dog label/clean/test'

# num_train_files = len([f for f in os.listdir(train_dir) if f.endswith(('.jpg', '.jpeg', '.png'))])
# num_test_files = len([f for f in os.listdir(test_dir) if f.endswith(('.jpg', '.jpeg', '.png'))])

# print(f"Number of training images: {num_train_files}")
# print(f"Number of test images: {num_test_files}")

Number of training images: 15508
Number of test images: 3876
